<a href="https://colab.research.google.com/github/Lizatol/Liza-Tolmacheva-1/blob/main/Accessibility_of_Healthcare_Facilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install osmnx mapclassify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 500.6 kB/s eta 0:00:00


In [2]:
!pip install spopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 19.3 MB/s eta 0:00:00


In [3]:
pip install --upgrade spopt

In [4]:
!pip install pyomo
!apt-get install -y coinor-cbc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 34 not upgraded.
Need to get 2,908 kB of archives.
After this operation, 8,310 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcoinutils3v5 amd64 2.11.4+repack1-2 [465 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libosi1v5 amd64 0.108.6+repack1-2 [275 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libclp1 amd64 1.17.5+repack1-1 [937 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcgl1 amd64 0.60.3+repack1-3 [420 kB]
Get:5 ht

In [ ]:
# Импорт библиотек
import osmnx as ox
import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
from shapely.geometry import Point, LineString
from sklearn.cluster import KMeans
from tqdm import tqdm
from scipy.spatial import distance_matrix
from spopt.locate import MCLP

# 1.
place_name = "Тихвинский район, Ленинградская область, Россия"
boundary = ox.geocode_to_gdf(place_name)
boundary_polygon = boundary.geometry.iloc[0]

# здания, мед. учреждения, дороги и природные барьеры
buildings = ox.features_from_polygon(boundary_polygon, tags={"building": ["apartments", "house", "residential"]})
health = ox.features_from_polygon(boundary_polygon, tags={"amenity": ["hospital", "clinic", "doctors", "health_post"]})
forests = ox.features_from_polygon(boundary_polygon, tags={"natural": ["scrub", "wood"]})
waterways = ox.features_from_polygon(boundary_polygon, tags={"natural": "water"})

# 2. Графы для маршрутизации
G_walk = ox.graph_from_polygon(boundary_polygon, network_type='walk')
G_drive = ox.graph_from_polygon(boundary_polygon, network_type='drive')

# 3. Подготовка геометрии и центроидов
buildings = gpd.GeoDataFrame(buildings, geometry='geometry', crs="EPSG:4326")
health = gpd.GeoDataFrame(health, geometry='geometry', crs="EPSG:4326")

# проекция вручную
projected_crs = "EPSG:32636"
buildings = buildings.to_crs(projected_crs)
health = health.to_crs(projected_crs)

# центроиды
buildings['centroid'] = buildings.geometry.centroid
buildings = buildings[buildings['centroid'].apply(lambda x: isinstance(x, Point))].copy()
health['centroid'] = health.geometry.centroid
health = health[health['centroid'].apply(lambda x: isinstance(x, Point))].copy()

# 4. Функции расчета времени и барьеров
def nearest_node(graph, point):
    return ox.nearest_nodes(graph, point.x, point.y)

def travel_time(graph, source_point, target_points, speed_kph):
    try:
        source = nearest_node(graph, source_point)
        times = []
        for target in target_points.geometry:
            dest = nearest_node(graph, target)
            length = nx.shortest_path_length(graph, source, dest, weight='length')
            time_min = length / (speed_kph * 1000 / 60)
            times.append(time_min)
        return min(times) if times else None
    except:
        return None

# 5. Расчет времени пешком и на авто
buildings = buildings.to_crs("EPSG:4326")  # временно для osmnx
health = health.to_crs("EPSG:4326")
buildings['walk_time_min'] = [travel_time(G_walk, pt, health, speed_kph=5) for pt in tqdm(buildings['centroid'].to_crs("EPSG:4326"))]
buildings['drive_time_min'] = [travel_time(G_drive, pt, health, speed_kph=40) for pt in tqdm(buildings['centroid'].to_crs("EPSG:4326"))]

# CRS обратно для расчётов
buildings = buildings.to_crs(projected_crs)
health = health.to_crs(projected_crs)

# 6. Нагрузка врачей
health['doctors'] = 3
health['capacity'] = 1000
health['load_factor'] = health['doctors'] / health['capacity']

# 7. Демография
buildings['residents'] = np.random.randint(1, 50, size=len(buildings))
buildings['elderly_share'] = np.random.uniform(0.1, 0.4, size=len(buildings))

# 8. Геобарьер: пересечение с маршрутами
forests = forests.to_crs(projected_crs)
waterways = waterways.to_crs(projected_crs)
barriers = gpd.GeoDataFrame(pd.concat([forests, waterways], ignore_index=True), crs=projected_crs)

def route_crosses_barrier(graph, origin, destination, barriers):
    try:
        path = nx.shortest_path(graph, source=origin, target=destination, weight='length')
        coords = [(graph.nodes[n]['x'], graph.nodes[n]['y']) for n in path]
        route_line = LineString(coords)
        return barriers.intersects(route_line).any()
    except:
        return True

# 9. Поиск зданий без пересечения барьеров

# Временно buildings в EPSG:4326
buildings_4326 = buildings.to_crs("EPSG:4326").copy()
health_4326 = health.to_crs("EPSG:4326").copy()

accessible_buildings = []
for idx, row in buildings_4326.iterrows():  # по перепроектированному DataFrame
    source = ox.nearest_nodes(G_walk, row['centroid'].x, row['centroid'].y)
    for dest_geom in health_4326.geometry:  # по перепроектированному GeoSeries
        dest = ox.nearest_nodes(G_walk, dest_geom.x, dest_geom.y)
        if not route_crosses_barrier(G_walk, source, dest, barriers):
            accessible_buildings.append(idx)
            break

# CRS обратно для расчётов
buildings = buildings_4326.to_crs(projected_crs) # buildings в исходную проекцию.
health = health_4326.to_crs(projected_crs)  # health в исходную проекцию.


buildings['barrier_free'] = buildings.index.isin(accessible_buildings)

# 10. Классификация доступности
buildings['access_mode'] = buildings.apply(lambda row:
    'walk' if row['walk_time_min'] is not None and row['walk_time_min'] <= 60 else
    'car' if row['drive_time_min'] is not None and row['drive_time_min'] <= 120 else
    'none', axis=1)

buildings['no_access'] = buildings['access_mode'] == 'none'
no_access_buildings = buildings[buildings['no_access']]

# 11. Оптимизация размещения новых объектов с учётом нагрузки врачей

# только здания, до которых можно добраться (без барьеров)
accessible_buildings = buildings[buildings['barrier_free']].copy()

# Точки спроса: центроиды зданий
demand_points = accessible_buildings['centroid'].to_crs("EPSG:4326")
demand_coords = np.array([[p.x, p.y] for p in demand_points])

# Точки предложения: центроиды учреждений
supply_points = health.geometry.centroid.to_crs("EPSG:4326")
supply_coords = np.array([[p.x, p.y] for p in supply_points])

# Матрица расстояний
cost_matrix = distance_matrix(demand_coords, supply_coords)

# Вес спроса — доля пожилых (влияние на приоритет)
weights = accessible_buildings['elderly_share'].values * 100

# Учёт нагрузки врачей как штрафа к расстоянию
load_penalty = health['load_factor'].values  # например, 0.003, 0.005 и т.п.
penalty_scale = 5  # масштаб штрафа (чем выше, тем больше избегаются перегруженные)
penalty_factors = 1 + (load_penalty * penalty_scale)

# Корректировка матрицы стоимости: штрафуем перегрузку
cost_matrix *= penalty_factors[np.newaxis, :]

# MCLP: размещаем 5 объектов с радиусом обслуживания 5 км (~0.05 градуса)
mclp = MCLP.from_cost_matrix(
    cost_matrix=cost_matrix,
    weights=weights,
    service_radius=0.05,  # приблизительно 5 км
    p_facilities=5
)
from pulp import PULP_CBC_CMD
mclp.solve(solver=PULP_CBC_CMD())

# Результаты
print("Дома без доступа к медицине:", len(no_access_buildings))
print("Оптимизированные местоположения учреждений (индексы в supply):", mclp.facility_client_array())

100%|██████████| 2178/2178 [08:38<00:00,  4.20it/s]


Дома без доступа к медицине: 2178
Оптимизированные местоположения учреждений (индексы в supply): None


In [ ]:
# 10. Классификация доступности
buildings['access_mode'] = buildings.apply(lambda row:
    'walk' if row['walk_time_min'] is not None and row['walk_time_min'] <= 60 else
    'car' if row['drive_time_min'] is not None and row['drive_time_min'] <= 120 else
    'none', axis=1)

buildings['no_access'] = buildings['access_mode'] == 'none'
no_access_buildings = buildings[buildings['no_access']]

# 11. Оптимизация размещения новых объектов с учётом нагрузки врачей

# только здания, до которых можно добраться (без барьеров)
accessible_buildings = buildings[buildings['barrier_free']].copy()

# Точки спроса: центроиды зданий
demand_points = accessible_buildings['centroid'].to_crs("EPSG:4326")
demand_coords = np.array([[p.x, p.y] for p in demand_points])

# Точки предложения: центроиды учреждений
supply_points = health.geometry.centroid.to_crs("EPSG:4326")
supply_coords = np.array([[p.x, p.y] for p in supply_points])

# Матрица расстояний
cost_matrix = distance_matrix(demand_coords, supply_coords)

# Вес спроса — доля пожилых (влияние на приоритет)
weights = accessible_buildings['elderly_share'].values * 100

# Учёт нагрузки врачей как штрафа к расстоянию
load_penalty = health['load_factor'].values  # например, 0.003, 0.005 и т.п.
penalty_scale = 5  # масштаб штрафа (чем выше, тем больше избегаются перегруженные)
penalty_factors = 1 + (load_penalty * penalty_scale)

# Корректировка матрицы стоимости: штрафуем перегрузку
cost_matrix *= penalty_factors[np.newaxis, :]

# MCLP: размещаем 5 объектов с радиусом обслуживания 5 км (~0.05 градуса)
mclp = MCLP.from_cost_matrix(
    cost_matrix=cost_matrix,
    weights=weights,
    service_radius=0.01,  # приблизительно 5 км
    p_facilities=7
)
from pulp import PULP_CBC_CMD
mclp.solve(solver=PULP_CBC_CMD())

# Результаты
print("Дома без доступа к медицине:", len(no_access_buildings))
print("Оптимизированные местоположения учреждений (индексы в supply):", mclp.facility_client_array())

Дома без доступа к медицине: 2178
Оптимизированные местоположения учреждений (индексы в supply): None


In [ ]:
buildings.explore()

In [ ]:
health.explore()

In [ ]:
!pip install folium

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Центр карты
boundary_latlon = boundary.to_crs("EPSG:4326")
boundary_center = boundary_latlon.geometry.unary_union.centroid
center = [boundary_center.y, boundary_center.x]

facility_indices_raw = mclp.facility_client_array()
facility_indices = np.unique([i for i in facility_indices_raw.flatten() if i is not None])

# Убедимся, что supply_points — это обычный GeoSeries с integer-индексами
supply_points = supply_points.reset_index(drop=True)
new_sites = gpd.GeoSeries([supply_points.iloc[i] for i in facility_indices], crs="EPSG:4326")

# Создаем карту
m = folium.Map(location=center, zoom_start=13, tiles='CartoDB positron')

# Кластер зданий без доступа
no_access_cluster = MarkerCluster(name='Здания без доступа').add_to(m)
for pt in no_access_buildings['centroid'].to_crs("EPSG:4326"):
    folium.CircleMarker(
        location=[pt.y, pt.x],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7
    ).add_to(no_access_cluster)

# Существующие мед. учреждения
for pt in health.geometry.centroid.to_crs("EPSG:4326"):
    folium.CircleMarker(
        location=[pt.y, pt.x],
        radius=6,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.9,
        popup="Существующее учреждение"
    ).add_to(m)

# Предложенные новые учреждения
for pt in new_sites:
    folium.Marker(
        location=[pt.y, pt.x],
        icon=folium.Icon(color='green', icon='plus-sign'),
        popup="Предложенное учреждение"
    ).add_to(m)

# Граница Мурино
folium.GeoJson(
    boundary_latlon.geometry,
    name='Границы Мурино',
    style_function=lambda x: {"fillColor": "none", "color": "gray", "weight": 1}
).add_to(m)

folium.LayerControl().add_to(m)
m

<ipython-input-13-39784e82b2cb>:6: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  boundary_center = boundary_latlon.geometry.unary_union.centroid


AttributeError: 'NoneType' object has no attribute 'flatten'

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

# Здания
buildings.plot(ax=ax, color='lightgrey', markersize=1, label='Здания')

# Медицинские учреждения
health.plot(ax=ax, color='red', markersize=10, label='Мед. учреждения')

# Дома без доступа
no_access_buildings.plot(ax=ax, color='black', markersize=5, label='Нет доступа')

# Оптимальные местоположения (если их координаты есть)
optimized_locs = supply_points.iloc[mclp.facility_indices]
optimized_locs.plot(ax=ax, color='blue', markersize=15, label='Оптимальные учреждения', marker='x')

# Легенда и заголовок
plt.legend()
plt.title('Доступность медицинских учреждений — Мурино и окрестности')
plt.xlabel("Долгота")
plt.ylabel("Широта")
plt.grid(True)
plt.show()

In [ ]:
# Импорт библиотек
import osmnx as ox
import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
from shapely.geometry import Point, LineString
from sklearn.cluster import KMeans
from tqdm import tqdm
from scipy.spatial import distance_matrix
from spopt.locate import MCLP

# 1. Получаем данные OSM
place_name = "Всеволожский район, Россия"
boundary = ox.geocode_to_gdf(place_name)
boundary_polygon = boundary.geometry.iloc[0]

# Получаем здания, мед. учреждения, дороги и природные барьеры
buildings = ox.features_from_polygon(boundary_polygon, tags={"building": True})
health = ox.features_from_polygon(boundary_polygon, tags={"amenity": ["hospital", "clinic", "doctors", "health_post"]})
forests = ox.features_from_polygon(boundary_polygon, tags={"landuse": "forest"})
waterways = ox.features_from_polygon(boundary_polygon, tags={"waterway": True})

# 2. Графы для маршрутизации
G_walk = ox.graph_from_polygon(boundary_polygon, network_type='walk')
G_drive = ox.graph_from_polygon(boundary_polygon, network_type='drive')

# 3. Подготовка геометрии
buildings = gpd.GeoDataFrame(buildings, geometry='geometry', crs="EPSG:4326")
health = gpd.GeoDataFrame(health, geometry='geometry', crs="EPSG:4326")
buildings['centroid'] = buildings.geometry.centroid
buildings = buildings[buildings['centroid'].apply(lambda x: isinstance(x, Point))].copy()

# 4. Функции расчета времени в пути и барьеров
def nearest_node(graph, point):
    return ox.nearest_nodes(graph, point.x, point.y)

def travel_time(graph, source_point, target_points, speed_kph):
    try:
        source = nearest_node(graph, source_point)
        times = []
        for target in target_points.geometry:
            dest = nearest_node(graph, target)
            length = nx.shortest_path_length(graph, source, dest, weight='length')
            time_min = length / (speed_kph * 1000 / 60)
            times.append(time_min)
        return min(times) if times else None
    except:
        return None

# 5. Расчет времени пешком и на авто
buildings['walk_time_min'] = [travel_time(G_walk, pt, health, speed_kph=5) for pt in tqdm(buildings['centroid'])]
buildings['drive_time_min'] = [travel_time(G_drive, pt, health, speed_kph=40) for pt in tqdm(buildings['centroid'])]

# 6. Нагрузка врачей (пример)
health['doctors'] = 3
health['capacity'] = 1000
health['load_factor'] = health['doctors'] / health['capacity']

# 7. Демография
buildings['residents'] = np.random.randint(1, 50, size=len(buildings))
buildings['elderly_share'] = np.random.uniform(0.1, 0.4, size=len(buildings))

# 8. Геобарьер: пересечение с маршрутами
forests = forests.to_crs("EPSG:32636")
waterways = waterways.to_crs("EPSG:32636")
barriers = gpd.GeoDataFrame(pd.concat([forests, waterways], ignore_index=True), crs=forests.crs)

buildings = buildings.to_crs("EPSG:32636")
health = health.to_crs("EPSG:32636")

def route_crosses_barrier(graph, origin, destination, barriers):
    try:
        path = nx.shortest_path(graph, source=origin, target=destination, weight='length')
        coords = [(graph.nodes[n]['x'], graph.nodes[n]['y']) for n in path]
        route_line = LineString(coords)
        return barriers.intersects(route_line).any()
    except:
        return True

accessible_buildings = []
for idx, row in buildings.iterrows():
    source = ox.nearest_nodes(G_walk, row['centroid'].x, row['centroid'].y)
    for dest_geom in health.geometry:
        dest = ox.nearest_nodes(G_walk, dest_geom.x, dest_geom.y)
        if not route_crosses_barrier(G_walk, source, dest, barriers):
            accessible_buildings.append(idx)
            break
buildings['barrier_free'] = buildings.index.isin(accessible_buildings)

# 9. Классификация доступности
buildings['access_mode'] = buildings.apply(lambda row:
    'walk' if row['walk_time_min'] is not None and row['walk_time_min'] <= 20 else
    'car' if row['drive_time_min'] is not None and row['drive_time_min'] <= 15 else
    'none', axis=1)

buildings['no_access'] = buildings['access_mode'] == 'none'

# 10. Дома без доступа
no_access_buildings = buildings[buildings['no_access']]

# 11. Оптимизация размещения с учетом барьеров и пожилых
accessible_buildings = buildings[buildings['barrier_free']].copy()
demand_points = accessible_buildings['centroid'].to_crs("EPSG:4326")
supply_points = health.geometry.centroid.to_crs("EPSG:4326")

# Геометрия -> координаты
demand_coords = np.array([[p.x, p.y] for p in demand_points])
supply_coords = np.array([[p.x, p.y] for p in supply_points])

# Матрица стоимости
cost_matrix = distance_matrix(demand_coords, supply_coords)

# Веса — пожилые жители
weights = accessible_buildings['elderly_share'].values * 100

# Модель MCLP
mclp = MCLP.from_cost_matrix(
    cost_matrix=cost_matrix,
    demand_weights=weights,
    service_radius=0.05,  # Примерный радиус (в градусах)
    p_facilities=5
)
mclp.solve()

print("Дома без доступа к медицине:", len(no_access_buildings))
print("Оптимизированные местоположения учреждений (индексы в supply):", mclp.facility_client_array)

In [ ]:
from spopt.locate.util import make_cost_matrix

In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
from shapely.geometry import Point
from sklearn.cluster import KMeans
from tqdm import tqdm

In [ ]:
# Импорт библиотек
import osmnx as ox
import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
from shapely.geometry import Point, LineString
from sklearn.cluster import KMeans
from tqdm import tqdm
from spopt.locate import MCLP
from spopt.locate.util import make_cost_matrix

# 1. Получаем данные OSM
place_name = "Всеволожский район, Россия"
boundary = ox.geocode_to_gdf(place_name)
boundary_polygon = boundary.geometry.iloc[0]

# Получаем здания, мед. учреждения, дороги и природные барьеры
buildings = ox.features_from_polygon(boundary_polygon, tags={"building": True})
health = ox.features_from_polygon(boundary_polygon, tags={"amenity": ["hospital", "clinic", "doctors", "health_post"]})
forests = ox.features_from_polygon(boundary_polygon, tags={"landuse": "forest"})
waterways = ox.features_from_polygon(boundary_polygon, tags={"waterway": True})

# 2. Графы для маршрутизации
G_walk = ox.graph_from_polygon(boundary_polygon, network_type='walk')
G_drive = ox.graph_from_polygon(boundary_polygon, network_type='drive')

# 3. Подготовка геометрии
buildings = gpd.GeoDataFrame(buildings, geometry='geometry', crs="EPSG:4326")
health = gpd.GeoDataFrame(health, geometry='geometry', crs="EPSG:4326")
buildings['centroid'] = buildings.geometry.centroid
buildings = buildings[buildings['centroid'].apply(lambda x: isinstance(x, Point))].copy()

# 4. Функции расчета времени в пути и барьеров
def nearest_node(graph, point):
    return ox.nearest_nodes(graph, point.x, point.y)

def travel_time(graph, source_point, target_points, speed_kph):
    try:
        source = nearest_node(graph, source_point)
        times = []
        for target in target_points.geometry:
            dest = nearest_node(graph, target)
            length = nx.shortest_path_length(graph, source, dest, weight='length')
            time_min = length / (speed_kph * 1000 / 60)
            times.append(time_min)
        return min(times) if times else None
    except:
        return None

# 5. Расчет времени пешком и на авто
buildings['walk_time_min'] = [travel_time(G_walk, pt, health, speed_kph=5) for pt in tqdm(buildings['centroid'])]
buildings['drive_time_min'] = [travel_time(G_drive, pt, health, speed_kph=40) for pt in tqdm(buildings['centroid'])]

# 6. Нагрузка врачей (пример)
health['doctors'] = 3
health['capacity'] = 1000
health['load_factor'] = health['doctors'] / health['capacity']

# 7. Демография
buildings['residents'] = np.random.randint(1, 50, size=len(buildings))
buildings['elderly_share'] = np.random.uniform(0.1, 0.4, size=len(buildings))

# 8. Геобарьер: пересечение с маршрутами
forests = forests.to_crs("EPSG:32636")
waterways = waterways.to_crs("EPSG:32636")
barriers = gpd.GeoDataFrame(pd.concat([forests, waterways], ignore_index=True), crs=forests.crs)

buildings = buildings.to_crs("EPSG:32636")
health = health.to_crs("EPSG:32636")

def route_crosses_barrier(graph, origin, destination, barriers):
    try:
        path = nx.shortest_path(graph, source=origin, target=destination, weight='length')
        coords = [(graph.nodes[n]['x'], graph.nodes[n]['y']) for n in path]
        route_line = LineString(coords)
        return barriers.intersects(route_line).any()
    except:
        return True

accessible_buildings = []
for idx, row in buildings.iterrows():
    source = ox.nearest_nodes(G_walk, row['centroid'].x, row['centroid'].y)
    for dest_geom in health.geometry:
        dest = ox.nearest_nodes(G_walk, dest_geom.x, dest_geom.y)
        if not route_crosses_barrier(G_walk, source, dest, barriers):
            accessible_buildings.append(idx)
            break
buildings['barrier_free'] = buildings.index.isin(accessible_buildings)

# 9. Классификация доступности
buildings['access_mode'] = buildings.apply(lambda row:
    'walk' if row['walk_time_min'] is not None and row['walk_time_min'] <= 20 else
    'car' if row['drive_time_min'] is not None and row['drive_time_min'] <= 15 else
    'none', axis=1)

buildings['no_access'] = buildings['access_mode'] == 'none'

# 10. Дома без доступа
no_access_buildings = buildings[buildings['no_access']]

# 11. Оптимизация размещения с учетом barier-free зданий и пожилых
demand_points = buildings[buildings['barrier_free']].centroid.to_crs("EPSG:4326")
supply_points = health.centroid.to_crs("EPSG:4326")
weights = buildings.loc[buildings['barrier_free'], 'elderly_share'].values * 100
cost_matrix = make_cost_matrix(demand_points, supply_points)

mclp = MCLP(
    cost_matrix=cost_matrix,
    demand_weights=weights,
    service_radius=10,
    p_facilities=5
)
mclp.solve()

print("Дома без доступа к медицине:", len(no_access_buildings))
print("Оптимизированные местоположения учреждений:", mclp.facility_client_array)

In [ ]:
# Импорт библиотек
import osmnx as ox
import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
from shapely.geometry import Point, LineString
from sklearn.cluster import KMeans
from tqdm import tqdm
from spopt.locate import MCLP
from spopt.locate import LSCP

# 1. Получаем данные OSM
place_name = "Всеволожский район, Россия"
boundary = ox.geocode_to_gdf(place_name)
boundary_polygon = boundary.geometry.iloc[0]

# Получаем здания, мед. учреждения, дороги и природные барьеры
buildings = ox.features_from_polygon(boundary_polygon, tags={"building": True})
health = ox.features_from_polygon(boundary_polygon, tags={"amenity": ["hospital", "clinic", "doctors", "health_post"]})
forests = ox.features_from_polygon(boundary_polygon, tags={"landuse": "forest"})
waterways = ox.features_from_polygon(boundary_polygon, tags={"waterway": True})

# 2. Графы для маршрутизации
G_walk = ox.graph_from_polygon(boundary_polygon, network_type='walk')
G_drive = ox.graph_from_polygon(boundary_polygon, network_type='drive')

# 3. Подготовка геометрии
buildings = gpd.GeoDataFrame(buildings, geometry='geometry', crs="EPSG:4326")
health = gpd.GeoDataFrame(health, geometry='geometry', crs="EPSG:4326")
buildings['centroid'] = buildings.geometry.centroid
buildings = buildings[buildings['centroid'].apply(lambda x: isinstance(x, Point))].copy()

# 4. Функции расчета времени в пути и барьеров
def nearest_node(graph, point):
    return ox.nearest_nodes(graph, point.x, point.y)

def travel_time(graph, source_point, target_points, speed_kph):
    try:
        source = nearest_node(graph, source_point)
        times = []
        for target in target_points.geometry:
            dest = nearest_node(graph, target)
            length = nx.shortest_path_length(graph, source, dest, weight='length')
            time_min = length / (speed_kph * 1000 / 60)
            times.append(time_min)
        return min(times) if times else None
    except:
        return None

# 5. Расчет времени пешком и на авто
buildings['walk_time_min'] = [travel_time(G_walk, pt, health, speed_kph=5) for pt in tqdm(buildings['centroid'])]
buildings['drive_time_min'] = [travel_time(G_drive, pt, health, speed_kph=40) for pt in tqdm(buildings['centroid'])]

# 6. Нагрузка врачей (пример)
health['doctors'] = 3
health['capacity'] = 1000
health['load_factor'] = health['doctors'] / health['capacity']

# 7. Демография
buildings['residents'] = np.random.randint(1, 50, size=len(buildings))
buildings['elderly_share'] = np.random.uniform(0.1, 0.4, size=len(buildings))

# 8. Геобарьер: пересечение с маршрутами
forests = forests.to_crs("EPSG:32636")
waterways = waterways.to_crs("EPSG:32636")
barriers = gpd.GeoDataFrame(pd.concat([forests, waterways], ignore_index=True), crs=forests.crs)

buildings = buildings.to_crs("EPSG:32636")
health = health.to_crs("EPSG:32636")

def route_crosses_barrier(graph, origin, destination, barriers):
    try:
        path = nx.shortest_path(graph, source=origin, target=destination, weight='length')
        coords = [(graph.nodes[n]['x'], graph.nodes[n]['y']) for n in path]
        route_line = LineString(coords)
        return barriers.intersects(route_line).any()
    except:
        return True

accessible_buildings = []
for idx, row in buildings.iterrows():
    source = ox.nearest_nodes(G_walk, row['centroid'].x, row['centroid'].y)
    for dest_geom in health.geometry:
        dest = ox.nearest_nodes(G_walk, dest_geom.x, dest_geom.y)
        if not route_crosses_barrier(G_walk, source, dest, barriers):
            accessible_buildings.append(idx)
            break
buildings['barrier_free'] = buildings.index.isin(accessible_buildings)

# 9. Классификация доступности
buildings['access_mode'] = buildings.apply(lambda row:
    'walk' if row['walk_time_min'] is not None and row['walk_time_min'] <= 20 else
    'car' if row['drive_time_min'] is not None and row['drive_time_min'] <= 15 else
    'none', axis=1)

buildings['no_access'] = buildings['access_mode'] == 'none'

# 10. Дома без доступа
no_access_buildings = buildings[buildings['no_access']]

# 11. Оптимизация размещения с учетом barier-free зданий и пожилых
demand_points = buildings[buildings['barrier_free']].centroid.to_crs("EPSG:4326")
supply_points = health.centroid.to_crs("EPSG:4326")
weights = buildings.loc[buildings['barrier_free'], 'elderly_share'].values * 100
cost_matrix = load_target_cost_matrix(demand_points, supply_points)

mclp = MaximalCoveringLocation(
    cost_matrix=cost_matrix,
    demand_weights=weights,
    service_radius=10,
    p_facilities=5
)
mclp.solve()

print("Дома без доступа к медицине:", len(no_access_buildings))
print("Оптимизированные местоположения учреждений:", mclp.facility_client_array)

In [ ]:
place_name = "Ленинградская область, Россия"
boundary = ox.geocode_to_gdf(place_name)
boundary_polygon = boundary.geometry.iloc[0]

In [ ]:
buildings = ox.features_from_polygon(boundary_polygon, tags={"building": ["apartments", "house", "residential"]})
health = ox.features_from_polygon(boundary_polygon, tags={"amenity": ["hospital", "clinic", "doctors", "health_post"]})
forests = ox.features_from_polygon(boundary_polygon, tags={"landuse": "forest"})
waterways = ox.features_from_polygon(boundary_polygon, tags={"waterway": True})
roads = ox.features_from_polygon(boundary_polygon, tags={"building": ["apartments", "house", "residential"]})

In [ ]:
G_walk = ox.graph_from_polygon(boundary_polygon, network_type='walk')
G_drive = ox.graph_from_polygon(boundary_polygon, network_type='drive')

In [ ]:
buildings = gpd.GeoDataFrame(buildings, geometry='geometry', crs="EPSG:4326")
health = gpd.GeoDataFrame(health, geometry='geometry', crs="EPSG:4326")
buildings['centroid'] = buildings.geometry.centroid
buildings = buildings[buildings['centroid'].apply(lambda x: isinstance(x, Point))].copy()